In [1]:
import csv
import pandas as pd
import numpy as np
import sklearn

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
reg = linear_model.LinearRegression()

np.set_printoptions(threshold=500)
pd.set_option('display.max_rows', 500)

In [2]:
path = './data/game_data_public.MKM.PremierDraft.csv'
with open(path) as csvfile:
    base_df = pd.read_csv(csvfile)

In [3]:
df = pd.DataFrame(base_df)
df = df[df['draft_time'] > '2024-02-19']

In [21]:
all_columns

['expansion',
 'event_type',
 'draft_id',
 'draft_time',
 'game_time',
 'build_index',
 'match_number',
 'game_number',
 'rank',
 'opp_rank',
 'main_colors',
 'splash_colors',
 'on_play',
 'num_mulligans',
 'opp_num_mulligans',
 'opp_colors',
 'num_turns',
 'won',
 'opening_hand_A Killer Among Us',
 'drawn_A Killer Among Us',
 'tutored_A Killer Among Us',
 'deck_A Killer Among Us',
 'sideboard_A Killer Among Us',
 'opening_hand_Absolving Lammasu',
 'drawn_Absolving Lammasu',
 'tutored_Absolving Lammasu',
 'deck_Absolving Lammasu',
 'sideboard_Absolving Lammasu',
 'opening_hand_Aftermath Analyst',
 'drawn_Aftermath Analyst',
 'tutored_Aftermath Analyst',
 'deck_Aftermath Analyst',
 'sideboard_Aftermath Analyst',
 'opening_hand_Agency Coroner',
 'drawn_Agency Coroner',
 'tutored_Agency Coroner',
 'deck_Agency Coroner',
 'sideboard_Agency Coroner',
 'opening_hand_Agency Outfitter',
 'drawn_Agency Outfitter',
 'tutored_Agency Outfitter',
 'deck_Agency Outfitter',
 'sideboard_Agency Outfitt

In [4]:
all_columns = list(df.columns)
name_columns = [col for col in all_columns if 'drawn_' in col]
names = [n.split('_')[1] for n in name_columns]

name_gen = lambda x: [f'{x}_{name}' for name in names]

In [22]:
deck_counts

A Killer Among Us                 85120
Absolving Lammasu                 12763
Aftermath Analyst                 42906
Agency Coroner                     6452
Agency Outfitter                   7296
Agrus Kos, Spirit of Justice      13030
Airtight Alibi                    37163
Alley Assailant                   44761
Alquist Proft, Master Sleuth      10852
Analyze the Pollen                23530
Anzrag's Rampage                    125
Anzrag, the Quake-Mole             9436
Archdruid's Charm                 21504
Assassin's Trophy                  8603
Assemble the Players               8914
Audience with Trostani             2931
Aurelia's Vindicator              15032
Aurelia, the Law Above            25795
Auspicious Arrival               134968
Axebane Ferox                     26164
Barbed Servitor                    9139
Basilica Stalker                  51137
Behind the Mask                    3960
Benthic Criminologists            19531
Bishop of the Bloodstained          223


In [23]:
deck_df = pd.DataFrame(df[name_gen('deck')])
deck_df.columns = names

deck_counts = deck_df.sum()
deck_counts.name = 'game counts'
deck_win_counts = deck_df[df['won']].sum()

gpwr = deck_win_counts / deck_counts
gpwr.name = 'gpwr'

In [6]:
oh_df = df[name_gen('opening_hand')]
oh_df.columns = names

drawn_df = df[name_gen('drawn')]
drawn_df.columns = names

ih_df = oh_df + drawn_df
ns_df = deck_df - ih_df
ns_df = ns_df.apply(lambda S: S.apply(lambda x: max(0, x)))

In [7]:
cards_seen = ih_df.sum(axis=1)
cards_seen.name = 'cards_seen'

In [8]:
gih_counts = ih_df.sum()
gih_win_counts = ih_df[df['won']].sum()

ns_counts = ns_df.sum()
ns_win_counts = ns_df[df['won']].sum()

gihwr = gih_win_counts / gih_counts
gihwr.name = 'gihwr'

gnswr = ns_win_counts / ns_counts
gnswr.name = 'gnswr'

In [9]:
ihd = gihwr - gpwr
ihd.name = 'in-hand delta'

In [10]:
gwih = ih_df.div(cards_seen, axis=0) * 40

In [11]:
gwihwr = gwih[df['won']].sum() / gwih.sum()
gwihwr.name = 'gwihwr'

In [12]:
gwihd = gwihwr - gpwr
gwihd.name = 'game-weighted in-hand delta'

In [13]:
deck_adjacency = deck_df.transpose().dot(deck_df)
average_marginal_decklist = deck_adjacency / deck_adjacency.sum()
d1gpwr = gpwr.dot(average_marginal_decklist)

d1d = gpwr - d1gpwr
d1d.name = 'distance-one delta'

In [25]:
ihd_df = pd.DataFrame([gpwr, ihd, gwihd, d1d, gihwr, gnswr, deck_counts])
# ihd_df.to_csv('ihd_df_6apr24.csv')

In [15]:
ih_analysis_df = pd.concat([deck_df, df[['won']], pd.DataFrame(cards_seen)], axis=1)

In [16]:
bias_map = {}
variance_map = {}
mean_map = {}
deck_gihwr = {}
deck_gnswr = {}

for name in names:
    filtered_df = ih_analysis_df[deck_df[name]>0]
    cards_seen_ar =  np.array(filtered_df['cards_seen'], dtype=np.double).reshape(filtered_df.shape[0], 1)
    cns_ar = 40 - cards_seen_ar
    won_ar = np.array(filtered_df['won'], dtype=np.double).reshape(filtered_df.shape[0], 1)
    filtered_deck_df = np.array(deck_df[name], dtype=np.double)[deck_df[name]>0].reshape(filtered_df.shape[0], 1)
    
    reg.fit(cards_seen_ar, won_ar)
    
    variance_map[name] = np.var(cards_seen_ar)
    mean_map[name] = np.mean(cards_seen_ar)
    bias_map[name] = reg.coef_[0][0]
    deck_gihwr[name] = np.sum(won_ar * cards_seen_ar * filtered_deck_df) / np.sum(cards_seen_ar * filtered_deck_df)
    deck_gnswr[name] = np.sum(won_ar * cns_ar * filtered_deck_df) / np.sum(cns_ar * filtered_deck_df)

In [17]:
bias = pd.Series(bias_map, index=names, name='cs bias')
var = pd.Series(variance_map, index=names, name='cs var')
mean = pd.Series(mean_map, index=names, name='cs mean')
deck_gihwr = pd.Series(deck_gihwr, index=names, name='deck gihwr')
deck_gnswr = pd.Series(deck_gnswr, index=names, name='deck gnswr')
bias_df = pd.DataFrame([bias, var, mean, deck_gihwr, deck_gnswr]).transpose()

In [18]:
bias_df['deck ihd form'] = bias_df['cs bias'] * bias_df['cs var'] / bias_df['cs mean']

In [30]:
metrics_df = pd.concat([ihd_df.transpose(), bias_df], axis=1)

In [31]:
metrics_df['deck ihd meas'] = metrics_df['deck gihwr'] - metrics_df['gpwr']
metrics_df['deck ihdd'] = metrics_df['deck ihd form'] - metrics_df['deck ihd meas']

In [32]:
metrics_df.to_csv('data/metrics_df_10apr24.csv')

In [33]:
metrics_df['deck ihd meas'].loc[[
    'Goblin Maskmaker', 
    'On the Job', 
    'Agrus Kos, Spirit of Justice', 
    'Inside Source', 
    'Makeshift Binding',
    'Tunnel Tipster',
    'Aftermath Analyst',
    'They Went This Way',
    "Detective's Satchel",
    'Chalk Outline'    
]]

Goblin Maskmaker               -0.011453
On the Job                     -0.009962
Agrus Kos, Spirit of Justice   -0.008741
Inside Source                  -0.008718
Makeshift Binding              -0.006270
Tunnel Tipster                  0.002079
Aftermath Analyst               0.008579
They Went This Way              0.012142
Detective's Satchel             0.016166
Chalk Outline                   0.022859
Name: deck ihd meas, dtype: float64

In [25]:
(final_df['in-hand delta'] / (final_df['in-hand delta'] - final_df['deck ihd meas']) < 0).sum()

65

In [27]:
(final_df['in-hand delta'] / (final_df['in-hand delta'] - final_df['deck ihd meas']) > 2).sum() + \
(final_df['in-hand delta'] / (final_df['in-hand delta'] - final_df['deck ihd meas']) < 1/2).sum()

146

In [28]:
final_df['deck ihd meas'].sort_values()

Mass Hysteria                  -0.014915
Goblin Maskmaker               -0.011453
Leonin Relic-Warder            -0.010624
On the Job                     -0.009962
Ranger-Captain of Eos          -0.009852
Seasoned Consultant            -0.009809
Frantic Scapegoat              -0.009567
Neighborhood Guardian          -0.009558
Felonious Rage                 -0.009403
Marketwatch Phantom            -0.008958
Agrus Kos, Spirit of Justice   -0.008741
Lightning Helix                -0.008721
Inside Source                  -0.008718
Novice Inspector               -0.008626
Ghostly Prison                 -0.008491
Karlov Watchdog                -0.008299
Perimeter Enforcer             -0.008015
Due Diligence                  -0.007884
Warleader's Call               -0.007865
Case of the Gateway Express    -0.007837
Dog Walker                     -0.007792
Caught Red-Handed              -0.007271
Haazda Vigilante               -0.007197
Red Herring                    -0.007189
Auspicious Arriv

In [33]:
(gihwr - final_df['deck ihd meas']).sort_values(ascending=False)

Aurelia's Vindicator            0.686756
Izoni, Center of the Web        0.645229
Agrus Kos, Spirit of Justice    0.639227
Smuggler's Copter               0.638507
Crashing Footfalls              0.637252
Vein Ripper                     0.634888
Tolsimir, Midnight's Light      0.633071
Hide in Plain Sight             0.627272
Cryptic Coat                    0.622725
Ezrim, Agency Chief             0.622135
Vannifar, Evolved Enigma        0.610549
Teysa, Opulent Oligarch         0.610351
Doppelgang                      0.607934
Steamcore Scholar               0.605067
Aurelia, the Law Above          0.603712
Trostani, Three Whispers        0.602646
Ill-Timed Explosion             0.602384
Pyrotechnic Performer           0.597918
Wojek Investigator              0.596587
A Killer Among Us               0.595429
Novice Inspector                0.595052
Kellan, Inquisitive Prodigy     0.594322
Kaya, Spirits' Justice          0.594037
Torch the Witness               0.593919
Lightning Helix 